In [ ]:

%load_ext autoreload
%autoreload 2

from katana import remote
from katana.remote import export_data, import_data

client = remote.Client()
client.server_version

print("--")


#  Nothing going on here- left unchanged


In [ ]:

from config import hyperparams

input_config = hyperparams.load_input_config()
input_config


#  Nothing going on here- left unchanged


In [ ]:

from timeit import default_timer

# Import the module that uses Dask to import drug data
from src import dask_ingestion

graph = client.create_graph(num_partitions=input_config.num_partitions)


#  if input_config.use_train_rdg:
#     print(f"Import pretrained graph from: {input_config.trained_rdg_path}")
#     import_data.rdg(graph, input_config.trained_rdg_path)
#  else:
#     print("Generate the graph with data from source")
#     dask_ingestion.generate_deepcdr_graph(graph, input_config)

import_data.rdg(graph, input_config.trained_rdg_path)
#  dask_ingestion.generate_deepcdr_graph(graph, input_config)
    
print("--")


#  RDG import works, CSV fails
#

#  Can we start naming the created graphs ?  In my shared server, there are a ton of orphan graphs.
#


In [ ]:

##################################################


In [ ]:

from src import katana_pipeline

rec_pipeline = katana_pipeline.RecipePipeline(graph)
rec_pipeline.graph.schema().view()

print("--")


#  Nothing is output  (no schema is output; product bug ?)
#


In [ ]:

##################################################


In [ ]:

#  this is mine
#

from tabulate import tabulate

print("--")



In [ ]:

#  A bunch of diagnostics post load
#

#  List all node types with node counts

l_query  = """
   MATCH (n) 
   WITH DISTINCT LABELS(n) AS temp, COUNT(n) AS tempCnt
   UNWIND temp AS label
   RETURN label, SUM(tempCnt) AS cnt
   ORDER BY label
   """.format()

l_result = graph.query_unpaginated(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Sample output,
#
#     +----+-----------+-------+
#     |    | label     |   cnt |
#     |----+-----------+-------|
#     |  0 | CELL_LINE |   494 |
#     |  1 | DRUG      |   223 |
#     |  2 | GDSC      |   266 |
#     |  3 | GENE      |   719 |
#     +----+-----------+-------+


In [ ]:

#  Return properties/keys from all vertices

l_query  = """
   MATCH (n)
   RETURN DISTINCT LABELS(n) AS label, KEYS(n)
   """.format()

l_result = graph.query_unpaginated(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Sample output,
#
#     +----+---------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
#     |    | label         | KEYS(n)                                                                                                                                                                                                                                                                                                                                                                                                                                          |
#     |----+---------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
#     |  1 | ['GENE']      | ['id']                                                                                                                                                                                                                                                                                                                                                                                                                                           |
#     |  2 | ['GDSC']      | ['id']                                                                                                                                                                                                                                                                                                                                                                                                                                           |
#     |  7 | ['DRUG']      | ['Count', 'Name', 'PubCHEM', 'Sample Size', 'Synonyms', 'Target pathway', 'Targets', 'drug_embeddings', 'drug_id', 'id', 'smiles']                                                                                                                                                                                                                                                                                                               |
#     | 11 | ['DRUG']      | ['Count', 'Name', 'PubCHEM', 'Sample Size', 'Target pathway', 'Targets', 'drug_embeddings', 'drug_id', 'id', 'smiles']                                                                                                                                                                                                                                                                                                                           |
#     |  0 | ['CELL_LINE'] | ['CCLE_ID', 'Doubling.Time.Calculated.hrs', 'Gender', 'Hist_Subtype1', 'Hist_Subtype2', 'Hist_Subtype3', 'Histology', 'Name', 'PATHOLOGIST_ANNOTATION', 'Pathology', 'Site_Primary', 'Site_Subtype1', 'Site_Subtype2', 'Site_Subtype3', 'epigenomics_embeddings', 'genomics_embeddings', 'genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'mutRate', 'tcga_code', 'transcriptomics_embeddings', 'type', 'type_refined'] |
#     |  3 | ['CELL_LINE'] | ['CCLE_ID', 'Doubling.Time.Calculated.hrs', 'epigenomics_embeddings', 'genomics_embeddings', 'genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'mutRate', 'tcga_code', 'transcriptomics_embeddings', 'type', 'type_refined']                                                                                                                                                                                             |
#     |  4 | ['CELL_LINE'] | ['CCLE_ID', 'Gender', 'Hist_Subtype1', 'Hist_Subtype2', 'Hist_Subtype3', 'Histology', 'Name', 'PATHOLOGIST_ANNOTATION', 'Pathology', 'Site_Primary', 'Site_Subtype1', 'Site_Subtype2', 'Site_Subtype3', 'epigenomics_embeddings', 'genomics_embeddings', 'genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'mutRate', 'tcga_code', 'transcriptomics_embeddings', 'type', 'type_refined']                                 |
#     |  5 | ['CELL_LINE'] | ['CCLE_ID', 'Doubling.Time.Calculated.hrs', 'Gender', 'Hist_Subtype1', 'Hist_Subtype2', 'Hist_Subtype3', 'Histology', 'Name', 'PATHOLOGIST_ANNOTATION', 'Pathology', 'Site_Primary', 'Site_Subtype1', 'Site_Subtype2', 'Site_Subtype3', 'epigenomics_embeddings', 'genomics_embeddings', 'genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'tcga_code', 'transcriptomics_embeddings', 'type', 'type_refined']            |
#     |  6 | ['CELL_LINE'] | ['CCLE_ID', 'Doubling.Time.Calculated.hrs', 'Hist_Subtype1', 'Hist_Subtype2', 'Hist_Subtype3', 'Histology', 'Name', 'PATHOLOGIST_ANNOTATION', 'Pathology', 'Site_Primary', 'Site_Subtype1', 'Site_Subtype2', 'Site_Subtype3', 'epigenomics_embeddings', 'genomics_embeddings', 'genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'mutRate', 'tcga_code', 'transcriptomics_embeddings', 'type', 'type_refined']           |
#     |  8 | ['CELL_LINE'] | ['CCLE_ID', 'Hist_Subtype1', 'Hist_Subtype2', 'Hist_Subtype3', 'Histology', 'Name', 'PATHOLOGIST_ANNOTATION', 'Pathology', 'Site_Primary', 'Site_Subtype1', 'Site_Subtype2', 'Site_Subtype3', 'epigenomics_embeddings', 'genomics_embeddings', 'genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'mutRate', 'tcga_code', 'transcriptomics_embeddings', 'type', 'type_refined']                                           |
#     |  9 | ['CELL_LINE'] | ['CCLE_ID', 'Gender', 'Hist_Subtype1', 'Hist_Subtype2', 'Hist_Subtype3', 'Histology', 'Name', 'Pathology', 'Site_Primary', 'Site_Subtype1', 'Site_Subtype2', 'Site_Subtype3', 'epigenomics_embeddings', 'genomics_embeddings', 'genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'mutRate', 'tcga_code', 'transcriptomics_embeddings', 'type', 'type_refined']                                                           |
#     | 10 | ['CELL_LINE'] | ['CCLE_ID', 'Doubling.Time.Calculated.hrs', 'Gender', 'Hist_Subtype1', 'Hist_Subtype2', 'Hist_Subtype3', 'Histology', 'Name', 'Pathology', 'Site_Primary', 'Site_Subtype1', 'Site_Subtype2', 'Site_Subtype3', 'epigenomics_embeddings', 'genomics_embeddings', 'genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'mutRate', 'tcga_code', 'transcriptomics_embeddings', 'type', 'type_refined']                           |
#     | 12 | ['CELL_LINE'] | ['CCLE_ID', 'Hist_Subtype1', 'Hist_Subtype2', 'Hist_Subtype3', 'Histology', 'Name', 'Pathology', 'Site_Primary', 'Site_Subtype1', 'Site_Subtype2', 'Site_Subtype3', 'epigenomics_embeddings', 'genomics_embeddings', 'genomics_expression', 'genomics_methylation', 'genomics_mutation', 'id', 'mutRate', 'tcga_code', 'transcriptomics_embeddings', 'type', 'type_refined']                                                                     |
#     +----+---------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+


In [ ]:

#  List all vertice types with counts

l_query  = """
   MATCH (m)-[r]->(n) 
   WITH DISTINCT TYPE(r) AS temp, COUNT(r) AS tempCnt
   RETURN temp, tempCnt
   ORDER BY temp
   """.format()

l_result = graph.query_unpaginated(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Sample output,
#
#     +----+----------------------+-----------+
#     |    | temp                 |   tempCnt |
#     |----+----------------------+-----------|
#     |  0 | FOR_DRUG             |       238 |
#     |  1 | HAS_CELL_LINE        |    105403 |
#     |  2 | HAS_GENE_OBSERVATION |  18222436 |
#     |  3 | SPLIT                |     94314 |
#     +----+----------------------+-----------+


In [ ]:

#  List edge properties
#

l_query  = """
   MATCH (n) - [r] -> (m)
   RETURN DISTINCT TYPE(r) AS my_type, KEYS(r) AS my_keys
   """.format()

l_result = graph.query_unpaginated(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Sample output,
#
#  +----+----------------------+-----------------------------------------------+
#  |    | my_type              | my_keys                                       |
#  |----+----------------------+-----------------------------------------------|
#  |  0 | SPLIT                | ['label', 'split']                            |
#  |  1 | HAS_CELL_LINE        | ['observation']                               |
#  |  2 | FOR_DRUG             | []                                            |
#  |  3 | HAS_GENE_OBSERVATION | ['full_column_name', 'observation', 'source'] |
#  +----+----------------------+-----------------------------------------------+


In [ ]:

##################################################


In [ ]:


start_time = default_timer()

#  This comes from katana_pipeline.py
#
#  rec_pipeline.feature_generator()

#  The above does this for 3 edge types; 
#     HAS_EXPRESSION_OBSERVATION": "genomics_expression
#     HAS_METHYLATION_OBSERVATION": "genomics_methylation
#     HAS_MUTATION_OBSERVATION": "genomics_mutation
# 
#  By calling this,
#     preprocessing.set_cell_line_property(self.graph, u, v)
#
#  Then it does this,,
#       preprocessing.remove_null_cells(self.graph)
#       preprocessing.delete_single_node(self.graph)


##########################################################

#  Listing source ..
#

#  We wont execute this here, and instead run below with checks pre and post 
#

#  def set_cell_line_property(graph, edge_label: str, genomics_property: str):
#     graph.query(
#        f"""
#        MATCH (c:CELL_LINE)-[r:{edge_label}]->(g:GENE)
#        WITH c, r, g
#        ORDER by g.ID
#        WITH c, collect(r.observation) as {genomics_property}
#        SET c.{genomics_property} = {genomics_property}
#        """
#     )
#  
#  def delete_single_node(graph):
#     """Delete single node in a graph"""
#     graph.query(
#        """
#        MATCH (n)
#        WHERE NOT (n)--()
#        DELETE n
#        """
#     )
#  
#  def remove_null_cells(graph):
#     """Delete cells node with NULL genomics features"""
#     graph.query(
#        """
#        MATCH (c:CELL_LINE)
#        WHERE c.genomics_methylation IS NULL
#           OR c.genomics_expression IS NULL
#           OR c.genomics_mutation IS NULL
#        DETACH DELETE c
#        """
#     )
#  
#  print(f"***Took {default_timer() - start_time} seconds to generate the features.***")


In [ ]:

#  The above does this for 3 edge types; 
#     HAS_EXPRESSION_OBSERVATION": "genomics_expression
#     HAS_METHYLATION_OBSERVATION": "genomics_methylation
#     HAS_MUTATION_OBSERVATION": "genomics_mutation

#  The above edge types do not exist
#

def set_cell_line_property(graph, edge_label: str, genomics_property: str):
   l_return = graph.query_unpaginated(
      f"""
      
      MATCH (c:CELL_LINE)-[r:{edge_label}]->(g:GENE)
      WITH c, r, g
      ORDER by g.ID
      RETURN COUNT(*)
      
      // WITH c, collect(r.observation) as {genomics_property}
      // SET c.{genomics_property} = {genomics_property}
      
      """
   )
   print(l_return)

set_cell_line_property(graph, "HAS_EXPRESSION_OBSERVATION", "genomics_expression")
set_cell_line_property(graph, "HAS_METHYLATION_OBSERVATION", "genomics_methylation")
set_cell_line_property(graph, "HAS_MUTATION_OBSERVATION", "genomics_mutation")


In [ ]:

#  Checking delete_single_node ..
#

#  One disconnected node is found, deleted
#

l_query  = """

   MATCH (n)
   WHERE NOT (n)--()
   
   // RETURN COUNT(*)
   //  Returns 1
   
   RETURN n
   //  Returns
   //
   //     +----+-----------------+-------------+------------+--------------+
   //     |    |           id(n) | labels(n)   | n.id       | n datatype   |
   //     |----+-----------------+-------------+------------+--------------|
   //     |  0 | 281474976710990 | ['GENE']    | Unnamed: 0 | node         |
   //     +----+-----------------+-------------+------------+--------------+
   
   """.format()

l_result = graph.query_unpaginated(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))


##########################################################

#  Pre,
#

l_query  = """
   MATCH (n) 
   WITH DISTINCT LABELS(n) AS temp, COUNT(n) AS tempCnt
   UNWIND temp AS label
   RETURN label, SUM(tempCnt) AS cnt
   ORDER BY label
   """.format()

l_result = graph.query_unpaginated(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Sample output,
#
#     +----+-----------+-------+
#     |    | label     |   cnt |
#     |----+-----------+-------|
#     |  0 | CELL_LINE |   494 |
#     |  1 | DRUG      |   223 |
#     |  2 | GDSC      |   266 |
#     |  3 | GENE      |   719 |
#     +----+-----------+-------+

##########################################################

#  The actual method as written,
#

l_query  = """

   MATCH (n)
   WHERE NOT (n)--()
   DELETE n
   
   """.format()

l_result = graph.query_unpaginated(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))


##########################################################

#  Post,

l_query  = """
   MATCH (n) 
   WITH DISTINCT LABELS(n) AS temp, COUNT(n) AS tempCnt
   UNWIND temp AS label
   RETURN label, SUM(tempCnt) AS cnt
   ORDER BY label
   """.format()

l_result = graph.query_unpaginated(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Sample output,
#
#     +----+-----------+-------+
#     |    | label     |   cnt |
#     |----+-----------+-------|
#     |  0 | CELL_LINE |   494 |
#     |  1 | DRUG      |   223 |
#     |  2 | GDSC      |   266 |
#     |  3 | GENE      |   718 |
#     +----+-----------+-------+

print("--")


In [ ]:

#  Checking remove_null_cells
#

l_query  = """

   MATCH (c:CELL_LINE)
   WHERE c.genomics_methylation IS NULL
      OR c.genomics_expression IS NULL
      OR c.genomics_mutation IS NULL
   //DETACH DELETE c
   RETURN COUNT(*)
   
   """.format()

l_result = graph.query_unpaginated(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Sample results,
#
#     +----+------------+
#     |    |   COUNT(*) |
#     |----+------------|
#     |  0 |          0 |
#     +----+------------+

#  Since this deletes nothing, we'll leave it be 
#


In [ ]:

#  This is all read-only
#

stats = rec_pipeline.stats()
stats


In [ ]:
##########################################################

In [ ]:

#  Same as above, we'll split this out line by line
#
#  The only effective code is split_generator, which is listed below
#

#  start_time = default_timer()
#  
#  rec_pipeline.split_generator(input_config)
#  
#  print(f"***Took {default_timer() - start_time} seconds to generate the split.***")
#  
#  rec_pipeline.graph.schema().view()


##########################################################


#  From katana_pipeline.py
#

    @utils.disable_warnings
    def split_generator(self, input_hp) -> None:
        """Add split edges in the graph

        Args:
            input_hp: The graph input parameters.
        """

        #  As expected, initially this does nothing, because there is no edge with the value "SPLIT"
        #
        print("Deleting old split")
        split.delete_split(self.graph)
 
        #  We run this traversal below with a count
        #
        print("Generating split")
        split_df = pandas.concat(
            self.graph.query(
                """
            MATCH (a:DRUG)<-[:FOR_DRUG]-(:GDSC)-[u:HAS_CELL_LINE]->(c:CELL_LINE)
            RETURN u.label as label, c.tcga_code as tcga_code,
            a.id as drug_id, c.id as cell_line_id
            """
            )
        )
        split_df = split_df.sort_values(by=["drug_id", "cell_line_id", "label"]).reset_index(drop=True)
        df_split = split.generate_split(split_df, input_hp.random_state, input_hp.test_size)
        print("Upsert split on the graph")
        split.upsert_split(self.graph, df_split)
        
        
##########################################################


#  From split.py
#

def delete_split(graph):
    graph.query(
        """
        MATCH (:DRUG)-[u:SPLIT]->(:CELL_LINE)
        DELETE u
        """
    )

def generate_split(df, random_state, test_size):
    df_train, df_test = train_test_split( df, test_size=test_size, random_state=random_state, stratify=df["tcga_code"], shuffle=True, )
    df_train["split"], df_test["split"] = SplitType.TRAIN, SplitType.VAL
    df_split = pandas.concat([df_train, df_test], ignore_index=True)
    df_split = df_split.astype({"drug_id": "string", "cell_line_id": "string"})
    return df_split.drop(columns=["tcga_code"])

def upsert_split(graph, split):
    with DataFrameImporter(graph) as df_importer:
        df_importer.node_id_property_name("id")
        df_importer.edges_dataframe(
            split,
            source_id_space="DRUG",
            destination_id_space="CELL_LINE",
            source_column="drug_id",
            destination_column="cell_line_id",
            type="SPLIT",
        )
        df_importer.insert()

        

In [ ]:

#  Testing this code, running a count instead
#

#       split_df = pandas.concat(
#           self.graph.query(
#               """
#           MATCH (a:DRUG)<-[:FOR_DRUG]-(:GDSC)-[u:HAS_CELL_LINE]->(c:CELL_LINE)
#           RETURN u.label as label, c.tcga_code as tcga_code,
#           a.id as drug_id, c.id as cell_line_id
#           """
#           )
#       )


l_query  = """

   MATCH (a:DRUG)<-[:FOR_DRUG]-(:GDSC)-[u:HAS_CELL_LINE]->(c:CELL_LINE)
   RETURN COUNT(*)
   // RETURN u.label as label, c.tcga_code as tcga_code,
   //    a.id as drug_id, c.id as cell_line_id
   
   """.format()

l_result = graph.query_unpaginated(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Sample output
#
#     +----+------------+
#     |    |   COUNT(*) |
#     |----+------------|
#     |  0 |      94314 |
#     +----+------------+


In [45]:

import pandas

#  So we'll actually run the above
#

# split_df = pandas.concat(
split_df =  graph.query_unpaginated(
   """
   
   MATCH (a:DRUG)<-[:FOR_DRUG]-(:GDSC)-[u:HAS_CELL_LINE]->(c:CELL_LINE)
   
   RETURN u.label as label, c.tcga_code as tcga_code, a.id as drug_id, c.id as cell_line_id
   """
   )
# )

print(len(split_df))
    
split_df = split_df.sort_values(by=["drug_id", "cell_line_id", "label"]).reset_index(drop=True)

print(tabulate(split_df.head(5), headers='keys', tablefmt='psql'))


#  df_split = split.generate_split(split_df, input_hp.random_state, input_hp.test_size)


#  print("Upsert split on the graph")
#  split.upsert_split(self.graph, df_split)
        
        





          0/? [?op/s]

94314
+----+---------+-------------+-----------+----------------+
|    | label   | tcga_code   |   drug_id | cell_line_id   |
|----+---------+-------------+-----------+----------------|
|  0 |         | OV          |  10027278 | ACH-000001     |
|  1 |         | LAML        |  10027278 | ACH-000002     |
|  2 |         | LAML        |  10027278 | ACH-000004     |
|  3 |         | LAML        |  10027278 | ACH-000006     |
|  4 |         | COAD/READ   |  10027278 | ACH-000007     |
+----+---------+-------------+-----------+----------------+


In [ ]:
##########################################################

<a id='step-3'></a>
## AI Training Pipeline

<img src="images/pipeline_gnn.png" style="width: 1500px;"/>

### Initialize model training

<center><img src="./images/model.png" style="width:70%;"></center>

DeepCDR is a hybrid graph convolutional network model that predicts the IC50 value, which denotes the effectiveness of a drug in inhibiting the growth of a specific cancer cell line.

On the one hand, the graph neural network (GNN) takes the adjacent information of atoms in a drug into consideration by aggregating the features of neighboring atoms together. On the other hand, the subnetworks extract high-level features of cancer omics profiles from a certain cancer cell line (i.e., genomic data, transcriptomic data and epigenomic data). Then the high-level features of drug and multiple omics data get concatenated and fed to a 1-D
convolutional neural network.

### Load Model configuration

The model configuration defines the specifications of the model that will be used to train on the graph.

<div class="alert alert-info">
You can change the model configuration in the <a href="./config/hyperparams.py" class="alert-link">hyperparams.py</a> file. By default the model is using genomics, epigenomics and transcriptomics data.
</div>

<div class="alert alert-info">
You can change the model architecture in the <a href="./src/model.py" class="alert-link">model.py</a> file.
</div>

In [ ]:
model_config = hyperparams.load_model_config()
model_config

### Load Training configuration

The training configuration specify details of the training pipeline.

<div class="alert alert-info">
You can change the training configuration in the <a href="./config/hyperparams.py" class="alert-link">hyperparams.py</a> file.
</div>

In [ ]:
training_config = hyperparams.load_training_config()
training_config

### Training

Distributed training of the model. The trained model and its configuration will be saved in the `rec_pipeline` graph using the `experiment_name` graph property.

In [ ]:
start_time = default_timer()
validation_metric = rec_pipeline.train(model_config, training_config)
print(f"***Took {default_timer() - start_time} seconds to train the model.***")
print("Validation metric: ", validation_metric)

### Testing

Test the trained model on the test data and compare the results with other achitecture baselines. We compare three different metrics:
- [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient): a measure of linear correlation between two sets of data. A value closer to 1 is better.
- [Spearman correlation coefficient](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient): how well the relationship between two variables can be described using a monotonic function. A value closer to 1 is better.
- [Root mean square error (RMSE)](https://en.wikipedia.org/wiki/Root-mean-square_deviation): a measure of the differences between values predicted by a model and the values observed. A lower value is better.

In [ ]:
start_time = default_timer()
metrics, predictions, labels = rec_pipeline.test(training_config)
print(f"***Took {default_timer() - start_time} seconds to test the model.***")
metrics

### Plot Predictions

Create a scatter plot of the `IC50` predicted by the trained model in function of `IC50` real value.

In [ ]:
start_time = default_timer()
rec_pipeline.plot(labels, predictions)
print(f"***Took {default_timer() - start_time} seconds to plot figures.***")

<a id='step-4'></a>
## Trained model inference 

<img src="images/pipeline_inference.png" style="width: 1500px;"/>



In [ ]:
rec_pipeline.infer(training_config)

In [ ]:
bortezomib = "B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN=C2)(O)O"
cell_line = "ACH-000001"
rec_pipeline.infer(training_config, drug=bortezomib, cell_line=cell_line)

## Run trained model to save node embeddings

Save the trained model embeddings as a node property. `drug_embeddings`, `epigenomics_embeddings`, `genomics_embeddings` and `transcriptomics_embeddings` will be created and saved. Those embeddings can be use for a future downstream task with a different model.

In [ ]:
start_time = default_timer()
rec_pipeline.infer_embeddings(model_config)
print(f"***Took {default_timer() - start_time} seconds to save node embeddings.***")
rec_pipeline.graph.schema().view()

## Save Graph

Save the graph created to the bucket location named in the `save_graph_path` variable.

In [ ]:
if input_config.save_graph_path:
    export_data.rdg(graph, input_config.save_graph_path)